In [11]:
from zipfile import ZipFile
file_name = 'content/preprocessed_train.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall('content')
  print('Done')

Done


In [12]:
import pandas as pd
import numpy as np
import cv2
import os
import random
from tqdm import tqdm

In [13]:
input_csv = "content/rafdb/train_labels.csv"
input_dir = "content/preprocessed_train"
output_dir = "content/augmented"
output_csv = "content/augmented_labels.csv"

# Tạo thư mục nếu chưa có
os.makedirs(output_dir, exist_ok=True)

In [14]:
df = pd.read_csv(input_csv)

print(df.head())

              image_name  class
0  train_00362_anger.jpg      3
1  train_01580_anger.jpg      3
2  train_03621_anger.jpg      3
3  train_04216_anger.jpg      3
4  train_06336_anger.jpg      3


In [15]:
print("Is duplicated:", df.duplicated().any())

Is duplicated: False


In [16]:
# Đếm số mẫu theo class
class_counts = df['class'].value_counts().sort_index()
max_count = class_counts.max()

print("Số lượng mẫu theo class:")
for cls, count in class_counts.items():
    print(f" - Class {cls}: {count} ảnh")

Số lượng mẫu theo class:
 - Class 0: 3817 ảnh
 - Class 1: 1032 ảnh
 - Class 2: 1585 ảnh
 - Class 3: 564 ảnh
 - Class 4: 573 ảnh
 - Class 5: 224 ảnh
 - Class 6: 2019 ảnh


In [17]:
# Tính số augmentation mỗi ảnh cần cho từng class
aug_multipliers = {
    int(cls): max(round(max_count / count), 1) - 1
    for cls, count in class_counts.items()
}

print("Số ảnh tăng cường trên mỗi ảnh gốc:")
for cls, mult in aug_multipliers.items():
    print(f" - Class {cls}: x{mult}")

Số ảnh tăng cường trên mỗi ảnh gốc:
 - Class 0: x0
 - Class 1: x3
 - Class 2: x1
 - Class 3: x6
 - Class 4: x6
 - Class 5: x16
 - Class 6: x1


In [18]:
# Tăng cường ảnh
augmented_rows = []

def apply_geometric_transform(image, mode="random"):
    h, w = image.shape[:2]
    M = None

    if mode == "random":
        mode = random.choice(["flip", "rotate", "affine", "perspective", "translate"])

    if mode == "flip":
        flip_code = random.choice([-1, 0, 1])  # -1: cả hai, 0: dọc, 1: ngang
        transformed = cv2.flip(image, flip_code)

    elif mode == "rotate":
        angle = random.uniform(-30, 30)
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)

    elif mode == "affine":
        pts1 = np.float32([[0,0], [w-1,0], [0,h-1]])
        pts2 = pts1 + np.random.randint(-15, 15, pts1.shape).astype(np.float32)
        M = cv2.getAffineTransform(pts1, pts2)

    elif mode == "perspective":
        margin = 0.1
        src = np.float32([
            [w * margin, h * margin],
            [w * (1 - margin), h * margin],
            [w * margin, h * (1 - margin)],
            [w * (1 - margin), h * (1 - margin)]
        ])
        dst = src + np.random.uniform(-w * 0.1, w * 0.1, src.shape).astype(np.float32)
        M = cv2.getPerspectiveTransform(src, dst)

    elif mode == "translate":
        tx = int(random.uniform(-0.3 * w, 0.3 * w))
        ty = int(random.uniform(-0.3 * h, 0.3 * h))
        M = np.float32([[1, 0, tx], [0, 1, ty]])

    else:
        raise ValueError(f"Unknown mode: {mode}")

    # borderMode: Cách xử lý phần biên khi pixel sau biến đổi nằm ngoài vùng ảnh gốc
    # cv2.BORDER_CONSTANT: đệm bằng giá trị cố định (như màu đen).
    # cv2.BORDER_REFLECT: phản chiếu biên (như gương).
    # cv2.BORDER_REPLICATE: lặp lại pixel gần nhất.
    # cv2.BORDER_WRAP: cuộn lại từ đầu (rất hiếm dùng).

    # flags: Cách nội suy pixel mới
    # cv2.INTER_LINEAR: nội suy tuyến tính (thường dùng cho ảnh).
    # cv2.INTER_NEAREST: nội suy gần nhất (nhanh nhưng có thể mất chi tiết).
    # cv2.INTER_CUBIC: nội suy bậc ba (chất lượng cao hơn, chậm hơn).
    # cv2.INTER_LANCZOS4: nội suy Lanczos (chất lượng cao, chậm nhất).

    # transformed = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT)
    # transformed_image = cv2.warpAffine(image, M, (w, h), borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))

    if mode == "flip":
        pass # Đã xử lý ở trên
    elif mode == "perspective":
        transformed = cv2.warpPerspective(image, M, (w, h), borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))
    else:
        transformed = cv2.warpAffine(image, M, (w, h), borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))

    return transformed, mode


In [19]:
for idx, row in tqdm(df.iterrows(), total=len(df)):
    img_name = row['image_name']
    label = row['class']
    multiplier = aug_multipliers[label]

    img_path = os.path.join(input_dir, img_name)
    image = cv2.imread(img_path)
    if image is None:
        print(f"Không đọc được ảnh: {img_path}")
        continue

    for i in range(multiplier):
        aug_img, mode = apply_geometric_transform(image)
        new_name = f"{os.path.splitext(img_name)[0]}_aug{i}_{mode}.jpg"
        save_path = os.path.join(output_dir, new_name)
        cv2.imwrite(save_path, aug_img)

        augmented_rows.append({
            'image_name': new_name,
            'class': label
        })

# Gộp dữ liệu gốc + tăng cường
df_aug = pd.DataFrame(augmented_rows)
df_combined = pd.concat([df, df_aug], ignore_index=True)
df_combined.to_csv(output_csv, index=False)
print(f"Đã lưu augmented CSV: {output_csv}")

100%|██████████| 9814/9814 [03:40<00:00, 44.41it/s]


Đã lưu augmented CSV: content/augmented_labels.csv


### Kiểm tra dữ liệu sau khi tăng cường

In [20]:
sub_df = pd.read_csv("content/augmented_labels.csv")
print('Số mẫu dữ liệu ban đầu là: ', len(df))
print('Số mẫu dữ liệu mới là: ', len(sub_df))

Số mẫu dữ liệu ban đầu là:  9814
Số mẫu dữ liệu mới là:  26920


In [21]:
image_files_aug = [f for f in os.listdir("content/augmented") if f.endswith('.jpg')]
image_files = [f for f in os.listdir("content/preprocessed_train") if f.endswith('.jpg')]

print('Số ảnh ban đầu là: ', len(image_files))
print('Số ảnh mới là: ', len(image_files_aug))

Số ảnh ban đầu là:  9814
Số ảnh mới là:  17106


In [22]:
print('Số ảnh của mỗi class sau khi augmentation')
sub_df.value_counts(sub_df['class'])

Số ảnh của mỗi class sau khi augmentation


class
1    4128
6    4038
4    4011
3    3948
0    3817
5    3808
2    3170
Name: count, dtype: int64

In [23]:
!zip -r "content/augmented.zip" "content/augmented"

'zip' is not recognized as an internal or external command,
operable program or batch file.
